In [7]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
import vertexai
from vertexai.generative_models import GenerativeModel, ChatSession
vertexai.init(project="alert-passkey-420723", location="us-central1")
model=GenerativeModel(model_name="gemini-1.0-pro",system_instruction=" You are an intelligent programmer. You must complete the python function given to you by the user. And you must follow the format they present when giving your answer!"
)

In [ ]:
!conda create -n codex python=3.7
! conda activate codex
! git clone https://github.com/openai/human-eval
! pip install -e human-eval
# Set the working directory
%cd /content/human-eval
!pip install jsonlines

In [8]:
generation_config = {
    "temperature": 0.0,
}


In [9]:
from human_eval.data import write_jsonl, read_problems
import pandas as pd
import time
import jsonlines
def generate_one_completion(prompt):
  prompt_msg =  "You are an intelligent programmer. You must complete the python function given to you by the user. And you must follow the format they present when giving your answer!"+prompt
  response = model.generate_content(prompt_msg,generation_config=generation_config)
  return response.candidates[0].text



In [11]:
import pandas as pd
import time
import jsonlines
import numpy as np

if __name__ == '__main__':
    jsonObj = pd.read_json(path_or_buf="https://github.com/openai/human-eval/raw/master/data/HumanEval.jsonl.gz", lines=True)
    command = 0
    i = 0
    durationTimes = []
    with jsonlines.open('Gemini1.0-pro.jsonl', mode='w') as writer:
      with jsonlines.open('GeminiTimer.jsonl', mode='w') as timer:
        while i < 164:
            if command < 4:
                start_time = time.time()
                response = generate_one_completion(jsonObj["prompt"][i])
                duration = time.time() - start_time
                timer.write({jsonObj["task_id"][i]: duration})
                durationTimes.append(duration)
                response = response.replace("```python\n", "")
                response = response.replace("\n```", "")
                completion = {'task_id': jsonObj["task_id"][i], 'completion': response}
                writer.write(completion)
                i = i + 1
                command = command + 1
            else:
                command = 0
                time.sleep(60)
    print(np.sum(duration))
    print(np.average(duration))

2.950490713119507
2.950490713119507


In [18]:
!python3 human_eval/evaluate_functional_correctness.py /content/Gemini1.0-pro.jsonl

Reading samples...
164it [00:00, 15556.94it/s]
Running test suites...
100% 164/164 [00:02<00:00, 73.85it/s]
Writing results to /content/Gemini1.0-pro.jsonl_results.jsonl...
100% 164/164 [00:00<00:00, 36013.92it/s]
{'pass@1': 0.3475609756097561}


In [13]:
%cd /content/human-eval

/content/human-eval


In [17]:
!python3 human_eval/evaluate_functional_correctness.py output.jsonl

Reading samples...
164it [00:00, 21170.97it/s]
Running test suites...
100% 164/164 [00:04<00:00, 34.02it/s]
Writing results to output.jsonl_results.jsonl...
100% 164/164 [00:00<00:00, 35095.20it/s]
{'pass@1': 0.676829268292683}


In [16]:
import json


def replace_newlines(input_file, output_file):

  with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
      entry = json.loads(line)
      completion = entry["completion"].replace("```\n", "")
      entry["completion"] = completion
      # Write modified entry back to output file with newline
      outfile.write(json.dumps(entry) + '\n')


# Example usage
input_file = '/content/Gemini1.0-pro.jsonl'
output_file = 'output.jsonl'
replace_newlines(input_file, output_file)

print(f'Successfully processed {input_file} and wrote results to {output_file}')


Successfully processed /content/Gemini1.0-pro.jsonl and wrote results to output.jsonl
